In [ ]:
import torch
import cv2
import torch.nn as nn
import matplotlib.pyplot as plt
device = torch.device('mps')

<div style="font-family: 'Arial', sans-serif; font-size: 16px; color: #5D5C61; padding: 15px; border-radius: 10px;">
    <h3 style="color: #7395AE;">Helper Function to load images</h3>
    <p style="font-style: italic; color: #B1A296;">This function processes and loads images for model input</p>
</div>

In [2]:
import cv2

import torchvision.transforms as transforms

image_path = 'Test_Fake/00A0WLZE5X.jpg'

def get_image(path):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),  
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.float()),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet statistics
    ])

    img = cv2.imread(path)

    img_tensor = transform(img)

    img_tensor = img_tensor.unsqueeze(0)

    return img_tensor.to(device)

img_tnsr = get_image(image_path)
print(img_tnsr.shape)

torch.Size([1, 3, 224, 224])


<div style="font-family: 'Arial', sans-serif; font-size: 16px; padding: 15px;">
    <h3>Visualizing Convolutional Layer Activations</h3>
    <p>This code visualizes the activations of convolutional layers in a pre-trained ResNet-34 model when processing both real and fake images.</p>
</div>
<div style="font-family: 'Arial', sans-serif; font-size: 16px; padding: 15px;">
<h3>Key Steps:</h3>
<ol>
    <li><strong>Load the Model</strong>: A pre-trained ResNet-34 model is loaded.</li>
    <li><strong>Extract Convolutional Layers</strong>: The code iterates through the model's structure to identify and collect all convolutional layers.</li>
    <li><strong>Process Images</strong>: The code processes both a real and a fake image using the <code>get_image</code> function (defined above).</li>
    <li><strong>Generate Activations</strong>: For each image:
        <ul>
            <li>The image is passed through each convolutional layer sequentially.</li>
            <li>The output (activation) of each layer is collected.</li>
        </ul>
    </li>
    <li><strong>Visualize Feature Maps</strong>: For each layer's activation:
        <ul>
            <li>The code creates a grid of subplots.</li>
            <li>Each subplot displays one feature map from the layer's output.</li>
            <li>The feature maps are displayed using a grayscale colormap.</li>
        </ul>
    </li>
</ol>
</div>
<div style="font-family: 'Arial', sans-serif; font-size: 16px; padding: 15px;">
<h3>Purpose:</h3>
<p style="font-style: italic;">This visualization helps understand how the convolutional layers in the ResNet-34 model process and transform the input image at different stages. By comparing the activations for real and fake images, we can potentially identify patterns that the model uses to distinguish between them.</p>
</div>

In [ ]:
import torch
import cv2
import torch.nn as nn
import matplotlib.pyplot as plt

device = torch.device('mps')

model = torch.load('models/resnet_34.pkl').to(device)

model_children = list(model.children())
num_layers = 0
conv_layers = []
for child in model_children:
    for model in child.children():
        for layer in model.children():
            if isinstance(layer, nn.Conv2d):
                conv_layers.append(layer)
                num_layers += 1
            elif isinstance(layer, nn.Sequential):
                for sub in layer.children():
                    for convs in sub.children():
                        if isinstance(convs, nn.Conv2d):
                            conv_layers.append(convs)
                            num_layers += 1

image_path = 'Test_Fake/00A0WLZE5X.jpg'
img_tnsr = get_image(image_path)

results = [conv_layers[0](img_tnsr)]
for i in range(1, len(conv_layers)):
    results.append(conv_layers[i](results[-1]))

for i in range(len(results)):
    layer_viz = results[i].squeeze()
    num_feature_maps = layer_viz.size(0)

    num_cols = 8
    num_rows = (num_feature_maps + num_cols - 1) // num_cols
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 4 * num_rows))
    axes = axes.flatten()
    print(f'Layer: {i + 1}')
    for j, f in enumerate(layer_viz):
        if j < len(axes):
            axes[j].imshow(f.detach().cpu().numpy(), cmap='gray')
            axes[j].axis('off')
        else:
            break
    plt.show()
    plt.close()

## Captum

In [7]:
import os
import random
from captum.attr import IntegratedGradients
from captum.attr import visualization as viz
import torch

device = torch.device('mps')
model = torch.load('models/resnet_34.pkl').to(device)

# Real
folder_path_real = "Test_Real"
real_images = [folder_path_real + '/'+ f for f in os.listdir(folder_path_real) if f.endswith(('.jpg', '.jpeg', '.png'))]

folder_path_fake = "Test_Fake"
fake_images = [folder_path_fake + '/' + f for f in os.listdir(folder_path_fake) if f.endswith(('.jpg', '.jpeg', '.png'))]

In [ ]:
import cv2

import torchvision.transforms as transforms

image_path = 'deepfake dataset/Real/00022.jpg'

def get_image(path):
    # Define the transformations
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),  # Resize the image to 256x256
        transforms.CenterCrop(224),  # Crop the center 224x224 portion
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.float()),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet statistics
    ])

    # Load the image
    img = cv2.imread(path)

    # Apply the transformations
    img_tensor = transform(img)

    # Add a batch dimension
    img_tensor = img_tensor.unsqueeze(0)

    return img_tensor.to(device)  # Move the tensor to the desired device

# Get the preprocessed image tensor
img_tnsr = get_image(image_path)
print(img_tnsr.shape)

### Integrated Gradients
- Integrated Gradients is an attribution method used to explain the predictions of deep learning models. It assigns importance scores to each input feature (in this case, pixels of an image) based on how much they contribute to the model's output.

In [ ]:
ig = IntegratedGradients(model)

In [ ]:
import matplotlib.pyplot as plt

target_class = 0

if target_class == 0:
    for i,image_file in enumerate(fake_images):

        img_tnsr = get_image(image_file)
        
        baseline = torch.zeros_like(img_tnsr)
        
        attributions = ig.attribute(img_tnsr, baselines=baseline, target=target_class, n_steps=100)
        
        vis_img = viz.visualize_image_attr(
            attributions.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
            img_tnsr.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
            method='blended_heat_map',
            sign='positive',
            show_colorbar=True,
            cmap='inferno',
            title=f'Integrated Gradients for {image_file}',
            fig_size=(10, 10)
        )
        
        vis_img[0].savefig(f'ig_{i+1}_fake',bbox_inches='tight', pad_inches=0)

elif target_class == 1:
    for i,image_file in enumerate(real_images):
        img_tnsr = get_image(image_file)
        
        baseline = torch.zeros_like(img_tnsr)
        
        attributions = ig.attribute(img_tnsr, baselines=baseline, target=target_class, n_steps=100)
        
        vis_img = viz.visualize_image_attr(
            attributions.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
            img_tnsr.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
            method='blended_heat_map',
            sign='positive',
            show_colorbar=True,
            cmap='inferno',
            title=f'Integrated Gradients for {image_file}',
            fig_size=(10, 10)
        )
        
        vis_img[0].savefig(f'ig_{i+1}_real',bbox_inches='tight', pad_inches=0)

### Saliency
- The og technique, returns the gradients with respect to the inputs

In [ ]:
from captum.attr import Saliency

sal = Saliency(model)

In [ ]:
target_classes = [0,1]
for target_class in target_classes:
    if target_class == 0:
        for i,image_file in enumerate(fake_images):

            img_tnsr = get_image(image_file)
            
            attributions = sal.attribute(img_tnsr, target=target_class)
            
            vis_img = viz.visualize_image_attr(
                attributions.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                img_tnsr.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                method='blended_heat_map',
                sign='positive',
                show_colorbar=True,
                cmap='inferno',
                title=f'Integrated Gradients for {image_file}',
                fig_size=(10, 10)
            )
            
            vis_img[0].savefig(f'sal_{i+1}_fake',bbox_inches='tight', pad_inches=0)
            
    elif target_class == 1:
        for i,image_file in enumerate(real_images):
            img_tnsr = get_image(image_file)
            
            baseline = torch.zeros_like(img_tnsr)

            attributions = sal.attribute(img_tnsr, target=target_class)
            
            vis_img = viz.visualize_image_attr(
                attributions.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                img_tnsr.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                method='blended_heat_map',
                sign='positive',
                show_colorbar=True,
                cmap='inferno',
                title=f'Integrated Gradients for {image_file}',
                fig_size=(10, 10)
            )
            
            vis_img[0].savefig(f'sal_{i+1}_real',bbox_inches='tight', pad_inches=0)
        
    

### DeepLiftSHAP

In [ ]:
from captum.attr import DeepLiftShap

dl = DeepLiftShap(model)

In [ ]:
target_classes = [0,1]

for target_class in target_classes:
    if target_class == 0:
        for i,image_file in enumerate(fake_images):

            img_tnsr = get_image(image_file)
            
            baselines = torch.rand(20,3,224,224)
            
            attributions = dl.attribute(img_tnsr, baselines=baselines ,target=target_class)
            
            vis_img = viz.visualize_image_attr(
                attributions.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                img_tnsr.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                method='blended_heat_map',
                sign='positive',
                show_colorbar=True,
                cmap='inferno',
                title=f'Integrated Gradients for {image_file}',
                fig_size=(10, 10)
            )
            
            vis_img[0].savefig(f'DLSHAP_{i+1}_fake',bbox_inches='tight', pad_inches=0)
    elif target_class == 1:
        for i,image_file in enumerate(real_images):

            img_tnsr = get_image(image_file)
            
            baselines = torch.rand(20,3,224,224)
            
            attributions = dl.attribute(img_tnsr, baselines=baselines ,target=target_class)
            
            vis_img = viz.visualize_image_attr(
                attributions.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                img_tnsr.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                method='blended_heat_map',
                sign='positive',
                show_colorbar=True,
                cmap='inferno',
                title=f'Integrated Gradients for {image_file}',
                fig_size=(10, 10)
            )
            
            vis_img[0].savefig(f'DLSHAP_{i+1}_real',bbox_inches='tight', pad_inches=0)


### Input X Gradient
- A baseline approach for computing the attribution. It multiplies input with the gradient with respect to input

In [ ]:
from captum.attr import InputXGradient

ixg = InputXGradient(model)

In [ ]:
target_classes = [0,1]

for target_class in target_classes:
    if target_class == 0:
        for i,image_file in enumerate(fake_images):

            img_tnsr = get_image(image_file)
            
            attributions = ixg.attribute(img_tnsr ,target=target_class)
            
            vis_img = viz.visualize_image_attr(
                attributions.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                img_tnsr.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                method='blended_heat_map',
                sign='positive',
                show_colorbar=True,
                cmap='inferno',
                title=f'Integrated Gradients for {image_file}',
                fig_size=(10, 10)
            )
            
            vis_img[0].savefig(f'IXG_{i+1}_fake',bbox_inches='tight', pad_inches=0)
    elif target_class == 1:
        for i,image_file in enumerate(real_images):

            img_tnsr = get_image(image_file)
                        
            attributions = ixg.attribute(img_tnsr,target=target_class)
            
            vis_img = viz.visualize_image_attr(
                attributions.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                img_tnsr.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                method='blended_heat_map',
                sign='positive',
                show_colorbar=True,
                cmap='inferno',
                title=f'Integrated Gradients for {image_file}',
                fig_size=(10, 10)
            )
            
            vis_img[0].savefig(f'IXG_{i+1}_real',bbox_inches='tight', pad_inches=0)


### Feature Ablation
- A perturbation based approach to computing attribution, involving replacing each input feature with a given baseline / reference, and computing the difference in output.

In [ ]:
from captum.attr import FeatureAblation
device = torch.device('mps')
model.to(device)
featabl = FeatureAblation(model)

In [ ]:
target_classes = [0,1]

for target_class in target_classes:
    if target_class == 0:
        for i,image_file in enumerate(fake_images[:3]):

            img_tnsr = get_image(image_file).to(device)

            attributions = featabl.attribute(img_tnsr ,target = target_class,show_progress = True,perturbations_per_eval = 25)
            
            vis_img = viz.visualize_image_attr(
                attributions.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                img_tnsr.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                method='blended_heat_map',
                sign='positive',
                show_colorbar=True,
                cmap='inferno',
                title=f'Integrated Gradients for {image_file}',
                fig_size=(10, 10)
            )
            
            vis_img[0].savefig(f'featabl_{i+1}_fake',bbox_inches='tight', pad_inches=0)
    elif target_class == 1:
        for i,image_file in enumerate(real_images[:3]):

            img_tnsr = get_image(image_file).to(device)
                        
            attributions = featabl.attribute(img_tnsr,target = target_class,show_progress = True,perturbations_per_eval = 25)
            
            vis_img = viz.visualize_image_attr(
                attributions.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                img_tnsr.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                method='blended_heat_map',
                sign='positive',
                show_colorbar=True,
                cmap='inferno',
                title=f'Integrated Gradients for {image_file}',
                fig_size=(10, 10)
            )
            
            vis_img[0].savefig(f'featabl_{i+1}_real',bbox_inches='tight', pad_inches=0)